## 1. 모델 이해하기
---
우리가 만들 모델의 아키텍처를 이해해본다. 합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 조금 다르다.<br><br>
### 1. 첫번째 표기 방법
합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스 풀링(nn.MaxPoold2d)은 풀링 층으로 별도 명명한다.<br><br>
### 2. 두번째 표기 방법
합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 본다.<br><br>
다시 말해 풀링도 하나의 층으로 보느냐, 안 보느냐의 문제인데 누가 옳고 틀리냐의 문제는 아니므로, 이번 챕터에서는 편의를 위해 맥스 풀링까지도 포함해서 하나의 합성곱 층으로 판단하고 정리해본다. 다시 말해 두번째 표기 방법을 택한다.<br><br>

## 2. 모델 구현하기

### 1. 필요한 도구 임포트와 입력의 정의

In [25]:
import torch
import torch.nn as nn

임의의 텐서를 만든다. 텐서의 크기는 1 x 1 x 28 x 28이다.

In [26]:
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


### 2. 합성곱층과 풀링 선언하기
이제 첫번째 합성곱 층을 구현해본다. 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1이다.

In [27]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이제 두번째 합성곱 층을 구현해본다. 32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1이다.

In [28]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


이제 맥스풀링을 구현해본다. 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정된다.

In [29]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


### 3. 구현체를 연결하여 모델 만들기
지금까지는 선언만한 것이고 아직 이들을 연결시키지는 않았다. 이들을 연결시켜서 모델을 완성시켜본다. 우선 입력을 첫번째 합성곱층을 통과시키고 합성곱층을 통과시킨 후의 텐서의 크기를 본다.

In [30]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널의 28너비 28높이의 텐서가 되었다. 32가 나온 이유는 conv1의 out_channel로 32를 지정해주었기 때문이다. 또한, 28너비 28높이가 된 이유는 패딩을 1폭으로 하고 3 x 3 커널을 사용하면 크기가 보존되기 때문이다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 본다.

In [31]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


32채널의 14너비 14높이의 텐서가 되었다. 이제 이를 다시 두번째 합성곱층에 통과시키고 통과한 후의 텐서의 크기를 본다.

In [32]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


64채널의 14너비 14높이의 텐서가 되었다 .64가 나온 이유는 conv2의 out_channel로 64를 지정해주었기 때문이다. 또한, 14너비 14높이가 된 이유는 패딩을 1폭으로 하고 3 x 3 커널을 사용하면 크기가 보존되기 때문이다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 본다. 이제 이를 맥스풀링을 통과시키고 맥스풀링을 통과한 후의 텐서의 크기를 본다.

In [33]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


이제 이 텐서를 펼치는 작업을 한다. 그런데 펼치기에 앞서 텐서의 n번째 차원을 접근하게 해주는 .size(n)에 대해서 배워본다. 현재 out의 크기는 1 x 64 x 7 x 7이다. out의 첫번째 차원이 몇인지 출력해본다.

In [34]:
out.size(0)

1

out의 첫번째 차원은 1이다. 나머지 차원이 몇인지 출력해본다.

In [35]:
out.size(1)

64

In [36]:
out.size(2)

7

In [37]:
out.size(3)

7

이제 이를 가지고 .view()를 사용하여 텐서를 펼치는 작업을 해본다.

In [38]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합된 것을 볼 수 있다. 이제 이에 대해서 전결합층(Fully-Connected layer)를 통과시켜본다. 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환한다.

In [39]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


## 3. CNN으로 MNIST 분류하기

In [40]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [41]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [43]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [50]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [45]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [46]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   
        out = self.fc(out)
        return out

In [47]:
model = CNN().to(device)

In [48]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [49]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [51]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
    
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225663364
[Epoch:    2] cost = 0.0630790442
[Epoch:    3] cost = 0.0462090448
[Epoch:    4] cost = 0.0371714309
[Epoch:    5] cost = 0.0308294613
[Epoch:    6] cost = 0.0253169481
[Epoch:    7] cost = 0.0210582539
[Epoch:    8] cost = 0.0176130496
[Epoch:    9] cost = 0.0157723483
[Epoch:   10] cost = 0.0130445706
[Epoch:   11] cost = 0.00996319856
[Epoch:   12] cost = 0.00945368875
[Epoch:   13] cost = 0.00808384456
[Epoch:   14] cost = 0.00628022524
[Epoch:   15] cost = 0.00676187035


In [53]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9868000149726868
